##이 코드의 주요 목적

1. 자연어 문장을 토큰화 한뒤 [CLS] 토큰의 임베딩 추출:

    BERT와 같은 트랜스포머 모델의 경우, [CLS] 토큰은 전체 문장을 대표하는 임베딩으로 사용됩니다. 코드는 모델이 각 문장을 처리한 후, 마지막 층에서 [CLS] 토큰의 임베딩 벡터를 추출하여 각 문장을 나타내는 벡터로 사용합니다.

2. t-SNE를 사용한 차원 축소:

    BERT와 같은 모델에서 추출된 [CLS] 임베딩은 일반적으로 수백 개의 차원을 가집니다(예: 이 코드에서는 1024차원). 하지만, 고차원 공간은 시각화가 어렵기 때문에 t-SNE (t-Distributed Stochastic Neighbor Embedding) 기법을 사용하여 2차원 또는 3차원으로 차원을 축소합니다.
    t-SNE는 고차원 데이터의 군집 구조를 저차원 공간에 투영하는 데 효과적인 방법입니다. 이를 통해 각 문장의 의미적 유사도를 시각적으로 확인할 수 있습니다.

3. 시각화 및 분석:

    t-SNE로 축소된 2차원 또는 3차원 데이터를 Plotly를 이용해 시각화하여, 각 문장(또는 데이터 포인트)이 어떻게 분포되어 있는지를 확인합니다.
    시각화를 통해 같은 레이블(또는 클래스)을 가진 문장들이 군집을 이루는지, 아니면 서로 다른 클래스가 혼재되어 있는지 등을 확인할 수 있습니다.
    이상치나 다른 군집의 특이한 점들을 시각적으로 발견할 수 있으며, 이를 통해 추가적인 분석(예: SHAP 분석 등)을 수행할 수도 있습니다.

# 환경 설정

In [1]:
!pip install transformers
##NLP에서 가장 많이 사용한 모듈 중 하나.
##다양한 사전 훈련된 언어 모델(BERT, GPT, T5 등)을 쉽게 불러오고 사용할 수 있게 해줌. 이 프로젝트에서는 chinese macbert large를 쓸거임
##BERT를 이용한 문장 임베딩 생성 && GPT-3을 이용한 문장 생성

!pip install keras_preprocessing
##keras에서 제공하는 데이터 전처리 유틸리티.
##이미지, 텍스트 등 다양한 형태의 데이터를 학습에 적합한 형식으로 변환해주는 기능
##텍스트 데이터->토큰 변환 && 이미지 데이터 리사이징, 회전, 스케일링
!pip install pad_sequences
##NLP 모델에서는 시퀀스가 길이가 다를 수 있음.
##모델 입력으로 데이터를 사용할 때 길이를 통일하기 위해 패딩을 사용
##변환된 텍스트 데이터를 CNN이나 LSTM같은 모델에 넣기 전 고정된 길이로 패딩 처리..
!pip install shap

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pad_sequences: filename=pad_sequences-0.6.1-py3-none-any.whl size=10199 sha256=319e29eb9ab5534f71e941019ec50d131bcda26f791b3a729c6fb2583571935f
  Stored in directory: /root/.cache/pip/wheels/48/9d/22/0a6305b87a9cc46ccc032060a041c3b59f39ac462f7358997e
Successfully built pad_sequences


In [2]:
##torch: PyTorch는 딥러닝 모델을 구축하고 학습시키기 위한 라이브러리
import torch

#numpy: 행렬 연산
import numpy as np

#Transformers: Hugging Face의 Transformers 라이브러리, BERT같은 사전 학습 모델을 제공.
#BertTokenizer: BERT모델에서 사용하는 토크나이저, 텍스트를 BERT 입력 형식에 맞게 변환
#BertForMaksedLM: MLM은 특정 단어가 가려졌을 때 해당 단어를 예측하는 모델
#BertForSequenceClassification: BERT 기반 문장 분류 모델을 불러오는 클래스, 분류 작업에 사용
#get_linear_schedule_with_warmup: 학습 스케쥴러, warmup 이후 선형적으로 학습률을 조절.

import transformers
from transformers import BertTokenizer, BertModel ##chinese-macbert-large
from transformers import BertForMaskedLM,BertForSequenceClassification
from transformers import get_linear_schedule_with_warmup
from transformers import AdamW, BertConfig

from tqdm import tqdm

##torch.nn.functional: 손실함수, 활성화 함수 등을 제공..
import torch.nn.functional as F

##pad_sequences: 입력 시퀀스의 길이를 맞추기 위한 함수. Keras에서 제공.
from keras_preprocessing.sequence import pad_sequences

##train_test_split: 데이터를 학습과 테스트 세트로 분할하기 위한 함수.
from sklearn.model_selection import train_test_split
##f1 score: 모델 평가 지표
from sklearn.metrics import f1_score
#defaultdict: 키가 없을 때도 에러를 발생하지 않고, 기본값을 반환해준다. 특별한 딕셔너리
from sklearn.manifold import TSNE


##TensorDataset: 데이터를 Tensor로 변환하여 PyTorch DataLoader에 적합한 형태로 만듬
##DataLoader: PyTorch에서 데이터 배치 처리를 위한 도구, 텐서를 batch 단위로 나누고 모델에 전달
##RandomSampler: DataLoader에서 데이터를 무작위로 샘플링하여 배치를 만들기 위한 클래스
##SequentialSampler: 순차데이터 샘플링
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

import random
import time
import datetime

import plotly.graph_objects as go
## 동적 시각화를 위한 plotly의 graph_objects
from collections import defaultdict

In [3]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print('No GPU available, using the CPU instead.')

There are 1 GPU(s) available.
We will use the GPU: NVIDIA A100-SXM4-40GB


# 전처리 과정

In [4]:
!gdown 1P-MbwJaKs0axA1Kn4ZEGTWk3Ml1j5iNe -O qilai_test_data.txt
!gdown 11qeIxlIJvZueHF3ajIJpmlxExwRhEJFt -O qilai_train_data.txt

Downloading...
From: https://drive.google.com/uc?id=1P-MbwJaKs0axA1Kn4ZEGTWk3Ml1j5iNe
To: /content/qilai_test_data.txt
100% 3.44k/3.44k [00:00<00:00, 13.9MB/s]
Downloading...
From: https://drive.google.com/uc?id=11qeIxlIJvZueHF3ajIJpmlxExwRhEJFt
To: /content/qilai_train_data.txt
100% 548k/548k [00:00<00:00, 136MB/s]


In [35]:
label_meaning = {
    0: 'Directional',  # 방향성 라벨
    1: 'Resultative',   # 결과 라벨
    2: 'Completive',    # 완료 라벨
    3: 'Inchoative',    # 시작 라벨
    4: 'Discourse'      # 담화 라벨
}
##라벨은 정답


##macbert large
label_list = [0, 1, 2, 3, 4]  # 라벨을 0~4로 맞추기


In [6]:
##파일의 문장에서 문장과 라벨을 분리하는 함수..
def load_data(filename):
    sentences=[]
    labels=[]

    with open(filename,'r') as f:
        for line in f.readlines():
            sen,lab=line.strip().split('\t')
            sentences.append(sen)
            labels.append(lab)

    return sentences,labels,len(sentences)


##문장의 mask를 new label로 바꾸는 전처리 함수
def preprocess(original_sentences,new_label):
    preprocessed_sentences=[]

    for sentence in original_sentences:
        preprocessed_sentences.append(sentence.replace('[MASK]',new_label)) ##new_label은 起来임

    return preprocessed_sentences



In [7]:
train = 'qilai_train_data.txt'
test = 'qilai_test_data.txt'

## 훈련 세트 문장, 라벨, 길이
train_sentences,train_labels,train_len=load_data(train)

## 테스트 세트 문장, 라벨, 길이
test_sentences,test_labels,test_len=load_data(test)

##라벨이 지금 str타입 1,2,3,4,5 이므로 0~4로 바꾸기!
train_labels_to_int=[int(label)-1 for label in train_labels]
test_labels_to_int=[int(label)-1 for label in test_labels]

##[MASK]를 제거한 문장들
train_sentences_mask_replaced=preprocess(train_sentences,'起来')
test_sentences_mask_replaced=preprocess(test_sentences,'起来')

# === 훈련 세트를 9:1 비율로 분할 (훈련 데이터 90%, 검증 데이터 10%) ===
train_sentences_split, validation_sentences, train_labels_split, validation_labels = train_test_split(
    train_sentences_mask_replaced, train_labels_to_int,
    test_size=0.1,  # 검증 데이터 비율: 10%
    random_state=42  # 랜덤 시드 고정 (재현성 보장)
)

# === 각 문장의 [MASK] 제거 후의 결과를 검증 세트에도 동일하게 적용 ===
# 검증 데이터 라벨도 동일하게 전처리한 결과로 유지
validation_labels = [int(label) for label in validation_labels]

# 결과 확인
print(f"전체 훈련 문장 개수: {train_len}")
print(f"훈련 세트 개수 (9/10): {len(train_sentences_split)}")
print(f"검증 세트 개수 (1/10): {len(validation_sentences)}")
print(f"테스트 세트 문장 개수: {test_len}")


전체 훈련 문장 개수: 5024
훈련 세트 개수 (9/10): 4521
검증 세트 개수 (1/10): 503
테스트 세트 문장 개수: 50


1. model  : 숫자를 학습
2. tokenizer : 자연어를 숫자로바꿔주고
3. pipeline : 고수준 API

# BertTokenizer 이용하기

텍스트 데이터를 처리하여 모델에 입력하기 위한 전처리 작업

1.  토큰화: 문장을 개별 단어 또는 토큰으로 분리
2.  패딩: 시퀀스 길이 고정을 위해 패딩 토큰 추가
3.  인코딩: 토큰을 고유한 숫자 인덱스로 변환
4.  어텐션 마스크 생성: 유효 토큰과 패딩 토큰을 구분하기 위해 마스크를 생성하는 과정

원 문장: 'ChatGPT is awesome'
1. ['ChatGPT', 'is', 'awesome']
2. ['ChatGPT', 'is', 'awesome', '[PAD]', '[PAD]']
3. [101, 2003, 1037, 0, 0]
4. [1, 1, 1, 0, 0]


In [8]:
##우리가 사용할 모델은 hugging face의 macbert
##아래 모델에서 쓸 수 있는 토크나이저 불러오기.
MODEL='hfl/chinese-macbert-large'
tokenizer=BertTokenizer.from_pretrained(MODEL)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/19.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/110k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/269k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/660 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [9]:
# === 훈련 세트에 대한 토큰화 작업 ===
train_sentences_tokenized = [tokenizer.tokenize(sen) for sen in train_sentences_split]
train_sentences_token_ids = pad_sequences(
    [tokenizer.convert_tokens_to_ids(token_sen) for token_sen in train_sentences_tokenized],
    dtype='long', truncating='post', padding='post'
)

# === 검증 세트에 대한 토큰화 작업 ===
validation_sentences_tokenized = [tokenizer.tokenize(sen) for sen in validation_sentences]
validation_sentences_token_ids = pad_sequences(
    [tokenizer.convert_tokens_to_ids(token_sen) for token_sen in validation_sentences_tokenized],
    dtype='long', truncating='post', padding='post'
)

# === 테스트 세트에 대한 토큰화 작업 ===
test_sentences_tokenized = [tokenizer.tokenize(sen) for sen in test_sentences_mask_replaced]
test_sentences_token_ids = pad_sequences(
    [tokenizer.convert_tokens_to_ids(token_sen) for token_sen in test_sentences_tokenized],
    dtype='long', truncating='post', padding='post'
)

print(f"훈련 데이터셋 크기: {len(train_sentences_token_ids)}")
print(f"검증 데이터셋 크기: {len(validation_sentences_token_ids)}")
print(f"테스트 데이터셋 크기: {len(test_sentences_token_ids)}")


훈련 데이터셋 크기: 4521
검증 데이터셋 크기: 503
테스트 데이터셋 크기: 50


# 어텐션 마스크 생성

In [10]:
# === 어텐션 마스크를 생성 ===
def create_attention_masks(token_ids):
    """토큰 ID에서 어텐션 마스크 생성하기"""
    return [[float(token_id > 0) for token_id in seq] for seq in token_ids]

# === 훈련 데이터에 대한 어텐션 마스크 생성 ===
train_attention_masks = create_attention_masks(train_sentences_token_ids)

# === 검증 데이터에 대한 어텐션 마스크 생성 ===
validation_attention_masks = create_attention_masks(validation_sentences_token_ids)

# === 테스트 데이터에 대한 어텐션 마스크 생성 ===
test_attention_masks = create_attention_masks(test_sentences_token_ids)

# 결과 확인
print(f"훈련 데이터 어텐션 마스크 크기: {len(train_attention_masks)}")
print(f"검증 데이터 어텐션 마스크 크기: {len(validation_attention_masks)}")
print(f"테스트 데이터 어텐션 마스크 크기: {len(test_attention_masks)}")



훈련 데이터 어텐션 마스크 크기: 4521
검증 데이터 어텐션 마스크 크기: 503
테스트 데이터 어텐션 마스크 크기: 50


torch.Tensor

In [11]:
import torch

# 데이터를 텐서로 변환하기
# 훈련 세트 90%
train_datas = torch.tensor(train_sentences_token_ids)  # 훈련 문장 데이터
train_labels = torch.tensor(train_labels_split)        # 훈련 라벨 데이터
train_masks = torch.tensor(train_attention_masks)      # 훈련 어텐션 마스크 데이터

# 검증 세트 10%
validation_datas = torch.tensor(validation_sentences_token_ids)  # 검증 문장 데이터
validation_labels = torch.tensor(validation_labels)              # 검증 라벨 데이터
validation_masks = torch.tensor(validation_attention_masks)      # 검증 어텐션 마스크 데이터

# 테스트 세트
test_datas = torch.tensor(test_sentences_token_ids)  # 테스트 문장 데이터
test_labels = torch.tensor(test_labels_to_int)       # 테스트 라벨 데이터
test_masks = torch.tensor(test_attention_masks)      # 테스트 어텐션 마스크 데이터

# 결과 확인 (각 데이터셋의 크기)
print(f"훈련 데이터 텐서 크기: {train_datas.size()}")
print(f"검증 데이터 텐서 크기: {validation_datas.size()}")
print(f"테스트 데이터 텐서 크기: {test_datas.size()}")


훈련 데이터 텐서 크기: torch.Size([4521, 145])
검증 데이터 텐서 크기: torch.Size([503, 124])
테스트 데이터 텐서 크기: torch.Size([50, 35])


In [12]:
# 배치 크기 설정
batch_size = 10

# === 훈련 데이터 ===
train_data = TensorDataset(train_datas, train_masks, train_labels)
train_sampler = RandomSampler(train_data)  # 무작위 샘플링
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

# === 검증 데이터 ===
validation_data = TensorDataset(validation_datas, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)  # 순차 샘플링
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

# === 테스트 데이터 ===
test_data = TensorDataset(test_datas, test_masks, test_labels)
test_sampler = SequentialSampler(test_data)  # 순차 샘플링
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)

# DataLoader의 크기 확인
print(f"훈련 DataLoader 크기: {len(train_dataloader)}")
print(f"검증 DataLoader 크기: {len(validation_dataloader)}")
print(f"테스트 DataLoader 크기: {len(test_dataloader)}")


훈련 DataLoader 크기: 453
검증 DataLoader 크기: 51
테스트 DataLoader 크기: 5


##학습 준비하기

In [13]:
model= BertForSequenceClassification.from_pretrained(MODEL,num_labels=5)
model.cuda()

pytorch_model.bin:   0%|          | 0.00/1.31G [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at hfl/chinese-macbert-large and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(21128, 1024, padding_idx=0)
      (position_embeddings): Embedding(512, 1024)
      (token_type_embeddings): Embedding(2, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-23): 24 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
              (LayerNorm): LayerNorm((1

In [14]:
#1. 옵티마이저 설정
## 모델의 가중치를 학습시키기 위해 사용하는 옵티마이저 설정
optimizer= AdamW(model.parameters(),lr=2e-5, eps=1e-8)

#2. 학습 에포크 수 설정
## 학습을 몇 번 반복할지 지정.
epochs=10

#3. 학습 단계수 계산
"""
   학습 전체에서 총 몇단계가 있는지 계산
   학습 단계 수=에포크 수×한 에포크 당 배치의 수
"""

total_batch=len(train_dataloader) * epochs

#4. 학습률 스케줄러 설정: 여기서는 선형 감소 스케줄러를 사용
"""
    선형 감소 스케줄러의 사용 이유
    1.빠른 초기 학습: 모델이 빠르게 큰 특징을 학습하기.
    2.세밀한 조정: 학습이 진행될 수록 학습률을 줄임으로써, 세밀한 특징까지 학습할 수 있게.
    3.모델의 안정성: 높은 학습률로 인한 진동과 발산 문제 방지
    4.스케줄링 전략은 다양함(step decay, exponential decay, cosine annealing, cyclic learning rate 등.)
"""

learning_scheduler=get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps=total_batch
)

#5. 재현성 보장을 위한 랜덤 시드 설정
seed_val=42
random.seed(seed_val)               # random의 난수 생성기 시드 설정: 데이터 전처리, 샘플링, 셔플링과 같은 작업의 일관성 보장
np.random.seed(seed_val)            # Numpy의 난수 생성기 시드 설정: Numpy 기반의 배열 연산, 샘플링, 셔플링과 같은 작업의 일관성 보장
torch.manual_seed(seed_val)         # PyTorch의 CPU 난수 생성기 시드 설정: CPU연산에서 난수를 생성하는 경우에 일관된 결과 보장
torch.cuda.manual_seed_all(seed_val)# Pytorch의 GPU 난수 생성기 시드 설정: GPU연산에서 난수를 생성하는 연산에서 일관된 결과 보장

#6. 정확도 계산 함수 정의
##classification 문제에서 모델의 예측 값과 정답 값을 비교하여 정확도를 계산.
def flat_accuracy(preds, labels):
    """
    모델의 예측값(preds)과 실제 라벨(labels)을 비교하여 정확도를 계산하다.

    Args:
        preds (numpy.ndarray or torch.Tensor): 모델의 예측값. 각 샘플에 대한 클래스 확률 또는 로짓값 배열.
        labels (numpy.ndarray or torch.Tensor): 실제 라벨 값.

    Returns:
        float: 예측 정확도 (0.0 ~ 1.0)
    """
    # 예측값에서 가장 높은 클래스 인덱스를 선택
    pred_flat=np.argmax(preds, axis=1).flatten()
    #실제 라벨을 평탄화
    labels_flat=labels.flatten()

    #예측값과 실제 라벨이 일치하는지 비교하여 정확도를 계산
    return np.sum(pred_flat==labels_flat) / len(labels_flat)

#7. 모델의 그래디언트 초기화
model.zero_grad()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


# tmi: 다양한 학습률 스케줄링 전략


In [15]:
# 1.   Step Decay (StepLR)

#     일정한 스텝마다 학습률을 특정 비율(gamma)로 줄인다

scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)

# 2.   Exponentail Decay (ExponentailLR)

#     학습률을 지수(gamma) 함수로 줄인다

scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.9)

# 3. Cosine Annealing (CosineAnnealingLR)

# 학습률을 코사인 함수의 형태로 조절하여, 학습이 끝날 때 천천히 줄어듦
# T_max는 학습률이 한 주기를 완성하는 단계 수.


scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=50)

# 4. Cyclic Learning Rate (CyclicLR)

# 학습률을 주기적으로 증가 및 감소시키는 전략
# step_size_up은 학습률이 증가하는 단계 수

scheduler = torch.optim.lr_scheduler.CyclicLR(optimizer, base_lr=0.001, max_lr=0.01, step_size_up=2000)

# 5. Reduce on Plateau (ReduceLROnPlateau)
# 성능이 patience동안 더 이상 개선되지 않을 때 학습률을 factor 비율로 줄이는 전략. mode는 성능 개선의 목표를 지정
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=10)

# 학습 시작

아래 코드의 주요 목적

[CLS] 토큰의 임베딩 추출:

**cls_embeddings = hidden_states[-1][:, 0, :].cpu().numpy()**

In [41]:
len(hidden_states)

25

In [16]:
# 훈련 및 검증 과정을 위한 코드
# === 딕셔너리 초기화 ===
loss_accuracy_metrics = defaultdict(list)  # loss와 accuracy를 저장할 딕셔너리
sentence_embeddings_per_epoch = []  # epoch별 임베딩 저장
all_epochs_labels = []  # 각 에포크의 라벨 저장
all_epochs_sentences = []  # 각 에포크의 문장 텍스트 저장

# === 시간 형식 변환 함수 ===
def format_time(elapsed):
    """시간을 'hh:mm:ss' 형식으로 변환"""
    return str(time.strftime("%H:%M:%S", time.gmtime(elapsed)))

# === 훈련 반복문 (전체 에포크 관리) ===
for epoch_i in range(epochs):
    print(f'\n======= Epoch {epoch_i + 1} / {epochs} =======')

    # === 훈련 단계 ===
    print("Training...")
    t0 = time.time()
    total_loss = 0
    model.train()

    # 훈련 데이터 로더의 각 배치에 대해 반복 수행
    for step, batch in enumerate(train_dataloader):
        if step % 500 == 0 and step > 0:
            elapsed = format_time(time.time() - t0)
            print(f'   Batch {step:,} of {len(train_dataloader):,}. Elapsed: {elapsed}.')

        # 배치를 각 데이터 요소별로 분리하여 GPU / CPU로 전송
        batch_token_ids, batch_mask, batch_labels = [t.to(device) for t in batch]

        # 모델에 입력 데이터를 전달하여 출력을 얻고, 손실 계산
        outputs = model(batch_token_ids, attention_mask=batch_mask, labels=batch_labels)
        loss = outputs[0]
        total_loss += loss.item()

        # 손실에 대해 역전파를 수행하여 그래디언트 계산
        loss.backward()

        # 그래디언트 클리핑으로 큰 값의 그래디언트를 제한 (폭발 방지)
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # 옵티마이저로 파라미터 업데이트, 스케줄러로 학습률 갱신, 모델의 그래디언트 초기화
        optimizer.step()
        learning_scheduler.step()
        model.zero_grad()

    # 에포크의 평균 훈련 손실 계산 및 저장
    avg_train_loss = total_loss / len(train_dataloader)
    loss_accuracy_metrics['train_loss'].append(avg_train_loss)
    print(f"  Average training loss: {avg_train_loss:.2f}")
    print(f"  Training epoch took: {format_time(time.time() - t0)}")

    # === 검증 단계 ===
    print('\nRunning Validation...')
    t0 = time.time()
    model.eval()

    eval_accuracy = 0
    now_epoch_embedding = []  # 현재 에포크의 검증 데이터 임베딩 저장
    now_epoch_label = []  # 현재 에포크의 라벨 저장
    now_epoch_text = []  # 현재 에포크의 문장 텍스트 저장

    # 검증 데이터 로더의 각 배치에 대해 반복 수행
    for batch in validation_dataloader:
        batch_token_ids, batch_mask, batch_labels = [t.to(device) for t in batch]

        # 그래디언트 계산하지 않도록 설정하고 모델에 입력 데이터를 전달하여 예측 수행
        with torch.no_grad():
            outputs = model(batch_token_ids, attention_mask=batch_mask, output_hidden_states=True)

        # 모델의 출력인 로짓을 NUMPY 배열로 변환
        logits = outputs.logits.detach().cpu().numpy()
        label_ids = batch_labels.to('cpu').numpy()

        # hidden_states에서 마지막 층의 [CLS] 토큰 임베딩 추출
        hidden_states = outputs.hidden_states
        eval_accuracy += flat_accuracy(logits, label_ids)

        # hidden_states가 존재하면 [CLS] 토큰의 임베딩 추출
        if hidden_states is not None:
            cls_embeddings = hidden_states[-1][:, 0, :].cpu().numpy()  # [CLS] 토큰의 임베딩 추출
            now_epoch_embedding.extend(cls_embeddings)
            now_epoch_label.extend(label_ids)
            now_epoch_text.extend([f"Sentence {i}" for i in range(len(label_ids))])
        else:
            print("hidden_states is None. Make sure output_hidden_states=True is set.")

    # 에포크의 평균 검증 정확도 계산 및 저장
    avg_eval_accuracy = eval_accuracy / len(validation_dataloader)
    loss_accuracy_metrics['val_accuracy'].append(avg_eval_accuracy)
    print(f"  Accuracy: {avg_eval_accuracy:.2f}")
    print(f"  Validation took: {format_time(time.time() - t0)}")

    # 현재 에포크의 임베딩, 라벨, 텍스트 저장
    sentence_embeddings_per_epoch.append(np.array(now_epoch_embedding))
    all_epochs_labels.append(np.array(now_epoch_label))
    all_epochs_sentences.append(now_epoch_text)

print('\nTraining complete!')


======= Epoch 1 / 10 =======
Training...
  Average training loss: 0.36
  Training epoch took: 00:01:31

Running Validation...
  Accuracy: 0.96
  Validation took: 00:00:02

======= Epoch 2 / 10 =======
Training...
  Average training loss: 0.15
  Training epoch took: 00:01:30

Running Validation...
  Accuracy: 0.97
  Validation took: 00:00:02

======= Epoch 3 / 10 =======
Training...
  Average training loss: 0.08
  Training epoch took: 00:01:30

Running Validation...
  Accuracy: 0.98
  Validation took: 00:00:02

======= Epoch 4 / 10 =======
Training...
  Average training loss: 0.04
  Training epoch took: 00:01:30

Running Validation...
  Accuracy: 0.97
  Validation took: 00:00:02

======= Epoch 5 / 10 =======
Training...
  Average training loss: 0.02
  Training epoch took: 00:01:30

Running Validation...
  Accuracy: 0.97
  Validation took: 00:00:02

======= Epoch 6 / 10 =======
Training...
  Average training loss: 0.01
  Training epoch took: 00:01:30

Running Validation...
  Accuracy: 0

# 테스트셋에 대한 스코어 검증

In [17]:
with torch.no_grad():
    outputs = model(test_datas.to(device), token_type_ids=None, attention_mask=test_masks.to(device))

test_result=[]
softmaxs=[]
logits=outputs[0]
logits = logits.detach().cpu()

for sen_logit in logits: ##문장별 softmax 함수 처리 -> 확률임.
    softmaxs.append(F.softmax(sen_logit, dim=0).numpy())

for i,sent in enumerate(test_sentences_mask_replaced):
    print(sent)
    logit=logits[i]
    predicted_num=label_list[np.argmax(logit)] # argmax -> 정답라벨임
    test_result.append(predicted_num)
    print(f'Predicted meaning is {predicted_num} ({label_meaning[predicted_num]})')

    for l,p in zip(label_list,softmaxs[i]):
        print(f"{str(l+1)}({label_meaning[l]}) : {p*100:.1f}")
    print()

随后,她把女儿抱起来,放到自己的腿上。
Predicted meaning is 0 (Directional)
1(Directional) : 100.0
2(Resultative) : 0.0
3(Completive) : 0.0
4(Inchoative) : 0.0
5(Discourse) : 0.0

这是一个大石块自然叠起来的洞子，一人多高，越往里越窄越低。
Predicted meaning is 2 (Completive)
1(Directional) : 0.0
2(Resultative) : 0.0
3(Completive) : 100.0
4(Inchoative) : 0.0
5(Discourse) : 0.0

他的话还没说完，看热闹的人都哗的一声大笑起来。
Predicted meaning is 3 (Inchoative)
1(Directional) : 0.0
2(Resultative) : 0.0
3(Completive) : 0.0
4(Inchoative) : 100.0
5(Discourse) : 0.0

小小笑起来，满口的小牙可白可整齐啦。
Predicted meaning is 4 (Discourse)
1(Directional) : 0.0
2(Resultative) : 0.1
3(Completive) : 0.0
4(Inchoative) : 0.3
5(Discourse) : 99.6

说完,站起来,作出宽怀大量的样子,一瘸一瘸走了。
Predicted meaning is 0 (Directional)
1(Directional) : 100.0
2(Resultative) : 0.0
3(Completive) : 0.0
4(Inchoative) : 0.0
5(Discourse) : 0.0

听完，他马上和老婆一样高兴起来。
Predicted meaning is 3 (Inchoative)
1(Directional) : 0.0
2(Resultative) : 0.0
3(Completive) : 0.0
4(Inchoative) : 100.0
5(Discourse) : 0.0

想到这个碴，小林兴奋起来，立即骑上车去找修车老头。

In [18]:
answer_list = []
wrong_result_list = []
y_true = []
y_pred = []

wrong = 0

wrongsents=[]
answernum=[]
wrongnum=[]

for i in range(len(test_labels)):
  if int(test_labels[i]) != int(test_result[i]):
    wrongsents.append(test_sentences_mask_replaced[i]); answernum.append(test_labels[i]); wrongnum.append(test_result[i])
    wrong += 1
    wrong_result_list.append(test_result[i]) # 오답 예측
    answer_list.append(test_labels[i]) # 정답 예측

  y_true.append(int(test_labels[i])) # 정답
  y_pred.append(int(test_result[i])) # 예측

print('1번(Directional)으로 잘못 예측한 경우 : ', wrong_result_list.count(0))
print('2번(Resultative)으로 잘못 예측한 경우 : ', wrong_result_list.count(1))
print('3번(Completive)으로 잘못 예측한 경우 : ', wrong_result_list.count(2))
print('4번(Inchoative)으로 잘못 예측한 경우 : ', wrong_result_list.count(3))
print('5번(Convenient)으로 잘못 예측한 경우 : ', wrong_result_list.count(4))


test_accuracy = (len(test_labels) - wrong) / len(test_labels) * 100
print('\nTotal Num : ', len(test_labels))
print('Correct : ', len(test_labels)-wrong)
print('Wrong : ', wrong)
print('\nTest Accuracy : ' , test_accuracy)

1번(Directional)으로 잘못 예측한 경우 :  0
2번(Resultative)으로 잘못 예측한 경우 :  0
3번(Completive)으로 잘못 예측한 경우 :  0
4번(Inchoative)으로 잘못 예측한 경우 :  0
5번(Convenient)으로 잘못 예측한 경우 :  2

Total Num :  50
Correct :  48
Wrong :  2

Test Accuracy :  96.0


지도학습

정답이 있는

회귀 // 분류

MSE, R2 score

강아지 vs 고양이
1 vs 2 vs 3

precision accuracy recall f1-score

In [ ]:
#

In [19]:
from sklearn import metrics

print(metrics.classification_report(y_true, y_pred))
print(metrics.confusion_matrix(y_true, y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        12
           2       1.00      1.00      1.00         4
           3       1.00      0.93      0.96        28
           4       0.75      1.00      0.86         6

    accuracy                           0.96        50
   macro avg       0.94      0.98      0.96        50
weighted avg       0.97      0.96      0.96        50

[[12  0  0  0]
 [ 0  4  0  0]
 [ 0  0 26  2]
 [ 0  0  0  6]]


#검증 데이터셋 500개에 대한 시각화

In [20]:
# 각 에포크마다 계산된 t-SNE임베딩을 저장할 리스트
tsne_embeddings_per_epoch=[]

for epoch_embeddings in sentence_embeddings_per_epoch: ##검증데이터셋에 대한 clsd토큰 값

    #2개의 차원으로 축소한 tsne 객체 생성
    tsne=TSNE(n_components=2, random_state=42)
    reduced_embeddings=tsne.fit_transform(epoch_embeddings)

    #축소된 임베딩을 리스트에 저장
    tsne_embeddings_per_epoch.append(reduced_embeddings)

## Plotly 시각화 작성하기.
epochs_range=list(range(1,epochs+1))

In [21]:
len(sentence_embeddings_per_epoch[0][0]), len(tsne_embeddings_per_epoch[0][0])

#차원이 1024에서 2로 줄은 것을 확인할 수 있다!

(1024, 2)

503

In [70]:
# 라벨의 의미를 정의 (각 라벨에 대한 설명)
label_meaning = {
    0: 'Directional',  # 방향성 라벨
    1: 'Resultative',   # 결과 라벨
    2: 'Completive',    # 완료 라벨
    3: 'Inchoative',    # 시작 라벨
    4: 'Discourse'      # 담화 라벨
}

# Plotly 시각화를 위한 Figure 객체 생성
fig = go.Figure()
'''plotly 시각화 만들기'''

# 각 라벨에 해당하는 색상 코드 정의 (Plotly 색상 팔레트에서 선택)
# https://colorhunt.co/ 에서 색상 참고하기!
label_colors = [
    '#636EFA',  # 라벨 0: 진한 파란색 (Blue)
    '#EF553B',  # 라벨 1: 주황색 (Orange-Red)
    '#00CC96',  # 라벨 2: 녹색 (Green)
    '#AB63FA',  # 라벨 3: 보라색 (Purple)
    '#FFA15A'   # 라벨 4: 밝은 주황색 (Light Orange)
]

# === 첫 번째 에포크의 시각화 ===
# 각 라벨에 대해 첫 에포크의 t-SNE 임베딩 데이터를 기반으로 산점도를 생성합니다.
for label in label_meaning.keys():
    # 현재 라벨에 해당하는 데이터 마스크 생성
    mask = np.array(all_epochs_labels[0]) == label

    # 각 라벨에 대해 산점도(Scatter) 생성
    scatter = go.Scatter(
        x=tsne_embeddings_per_epoch[0][mask, 0],  # 첫 번째 축의 임베딩 값
        y=tsne_embeddings_per_epoch[0][mask, 1],  # 두 번째 축의 임베딩 값
        mode='markers',  # 마커 모드로 설정

        marker=dict(
            size=8,  # 마커 크기
            color=label_colors[label],  # 각 라벨에 해당하는 색상 지정
            showscale=False  # 첫 에포크의 범례 제거
        ),
        # 전체 데이터의 순서를 유지하며 문장 번호 표시
        text=[f"{validation_sentences[idx]}" for idx in range(len(tsne_embeddings_per_epoch[0])) if mask[idx]],
        hoverinfo='text',  # 툴팁에 표시될 정보의 유형 설정
        name=f"{label_meaning.get(label)}",  # 범례에 표시할 라벨 이름 설정
        visible=True  # 첫 번째 에포크에서 보이도록 설정
    )
    fig.add_trace(scatter)  # 산점도를 그래프에 추가

# === 애니메이션 프레임 만들기 ===
# 각 에포크에 대해 애니메이션에 사용할 프레임 데이터를 생성합니다.
frames = []  # 각 에폭에 대한 scatter 정보를 담을 리스트
for epoch in range(epochs):
    frame_data = []  # 각 프레임에서 사용할 산점도 데이터를 저장할 리스트
    for label in label_meaning.keys():
        # 현재 에포크의 해당 라벨에 대한 데이터 마스크 생성
        mask = np.array(all_epochs_labels[epoch]) == label

        # 프레임의 각 라벨에 대해 산점도 생성
        frame_data.append(go.Scatter(
            x=tsne_embeddings_per_epoch[epoch][mask, 0],  # 첫 번째 축의 임베딩 값
            y=tsne_embeddings_per_epoch[epoch][mask, 1],  # 두 번째 축의 임베딩 값
            mode='markers',  # 마커 모드로 설정

            marker=dict(
                size=4,  # 마커 크기
                color=label_colors[label],  # 라벨에 해당하는 색상 지정
                showscale=False  # 범례 제거
            ),
            # 전체 데이터의 순서를 유지하며 문장 번호 표시
            text=[f"{validation_sentences[idx]}" for idx in range(len(tsne_embeddings_per_epoch[0])) if mask[idx]],
            hoverinfo='text',  # 툴팁에 표시될 정보의 유형 설정
            name=f"{label_meaning.get(label)}",  # 범례에 표시할 라벨 이름 설정
        ))
    # 현재 에포크에 대한 프레임 데이터 생성 및 추가
    frames.append(go.Frame(data=frame_data, name=f'Epoch {epoch + 1}'))

# === 애니메이션 컨트롤을 위한 슬라이더 만들기 ===
# 각 에포크에 해당하는 슬라이더 단계를 생성하여 사용자가 슬라이더로 에포크를 선택할 수 있도록 설정
sliders = [dict(
    steps=[dict(method='animate',
                args=[[f'Epoch {k + 1}'],
                      dict(mode='immediate',
                           frame=dict(duration=1500, redraw=True),  # 프레임 지속 시간 1500ms
                           transition=dict(duration=1000)          # 프레임 전환 시간 1000ms
                           )
                      ],
                label=f'Epoch {k + 1}') for k in range(epochs)],  # 각 에포크에 해당하는 슬라이더 단계를 설정
    active=0,  # 슬라이더의 기본 활성화 단계 설정
    transition=dict(duration=1000),  # 슬라이더 전환 시간 설정
    x=0.1, y=0, len=0.9,  # 슬라이더의 위치 및 길이 설정
    currentvalue=dict(font=dict(size=20), prefix="Epoch: ", visible=True, xanchor='center')
)]

# === 레이아웃 및 설정 ===
# 그래프의 제목, 애니메이션 버튼 및 슬라이더 설정
fig.update_layout(
    title='t-SNE Sentence Embeddings by Label with Animation',  # 그래프 제목 설정
    updatemenus=[dict(type="buttons",  # 애니메이션 재생/일시정지 버튼 생성
                      showactive=False,
                      buttons=[dict(label="Play",
                                    method="animate",
                                    args=[None, dict(frame=dict(duration=1500, redraw=True),
                                                     fromcurrent=True,
                                                     mode='immediate')]),
                               dict(label="Pause",
                                    method="animate",
                                    args=[[None], dict(frame=dict(duration=0, redraw=False),
                                                       mode='immediate')])])],

    sliders=sliders,  # 슬라이더 추가
    legend_title="Labels",  # 범례 제목 설정
    showlegend=True,  # 범례를 보이도록 설정
    xaxis=dict(range=[-50,50]),
    yaxis=dict(range=[-50,50])
)

# === 애니메이션 프레임 추가 ===
# 생성된 프레임을 그래프에 추가
fig.frames = frames

# 그래프를 출력
fig.show()


##검증 데이터셋의 이상치 분석

In [23]:
##이상치 인덱스 저장 >> 수작업
validation_outlier_indices=sorted([503,107,242,129,169,340,65,250,201,215,249])

#201빨강
#215빨강
#249 보라
validation_outlier_sentences=[validation_sentences[idx-1] for idx in validation_outlier_indices]

In [24]:
import shap
print(validation_sentences[201-1])
print(validation_sentences[215-1])
print(validation_sentences[249-1])


腹部的肿瘤胀得厉害，一年一年地胀起来，比怀小孩更辛苦。
再说，我生孩子以后就胖起来了，这样的运动能让自己有个好身材。
别看他平常软绵绵的象团棉花，可要硬起来就象块钢一样，比江水山还厉害。


In [25]:
prediction= transformers.pipeline(
    "text-classification",
    model=model, #
    tokenizer=tokenizer,
    device=0,
    return_all_scores=True,
)

explainer=shap.Explainer(prediction)
shap_values=explainer(validation_outlier_sentences)

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/text_classification.py:104: UserWarning:

`return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


  0%|          | 0/498 [00:00<?, ?it/s]

PartitionExplainer explainer:   9%|▉         | 1/11 [00:00<?, ?it/s]

  0%|          | 0/498 [00:00<?, ?it/s]

PartitionExplainer explainer:  27%|██▋       | 3/11 [00:20<00:36,  4.61s/it]

...

PartitionExplainer explainer:  91%|█████████ | 10/11 [01:22<00:08,  8.52s/it]

  0%|          | 0/498 [00:00<?, ?it/s]

PartitionExplainer explainer: 100%|██████████| 11/11 [01:30<00:00,  8.56s/it]

  0%|          | 0/498 [00:00<?, ?it/s]

PartitionExplainer explainer: 12it [01:39,  9.07s/it]


In [26]:


# 모델에 입력하여 각 문장에 대한 예측 결과를 분석
def analyze_outlier_probabilities(outlier_indices,dataloader,original_sentences,shap_values):

    # 모델을 평가 모드로 설정
    model.eval()

    # 이상치 인덱스의 예측 결과를 저장하기 위한 리스트
    outlier_predictions = []

    # 검증 데이터 로더의 각 배치에 대해 반복 수행
    for step, batch in enumerate(dataloader):
        batch_token_ids, batch_mask, batch_labels = [t.to(device) for t in batch]

        # 그래디언트 계산하지 않도록 설정하고 모델에 입력 데이터를 전달하여 예측 수행
        with torch.no_grad():
            outputs = model(batch_token_ids, attention_mask=batch_mask, output_hidden_states=True)

        # 모델의 출력인 로짓을 NUMPY 배열로 변환
        logits = outputs.logits.detach().cpu().numpy()
        batch_labels_np = batch_labels.to('cpu').numpy()

        # 각 배치의 문장을 순회하면서, 해당 문장이 이상치 인덱스에 있는지 확인
        for idx_in_batch, (logit, true_label) in enumerate(zip(logits, batch_labels_np)):
            global_idx = step * dataloader.batch_size + idx_in_batch + 1
            if global_idx in outlier_indices:
                # 해당 문장이 이상치라면, 소프트맥스 확률 계산 및 결과 저장
                softmax_probs = F.softmax(torch.tensor(logit), dim=0).numpy()
                predicted_class = np.argmax(softmax_probs)  # 0부터 시작하는 인덱스로 예측된 라벨

                outlier_predictions.append({
                    "global_idx": global_idx,
                    "sentence": original_sentences[global_idx - 1],
                    "true_label": int(true_label),
                    "predicted_label": predicted_class,
                    "probabilities": softmax_probs,
                })

    with open("起来보어 검증데이터에서의 이상치 분석.html", "a", encoding='UTF-8') as f:
        for i, prediction in enumerate(outlier_predictions):
            # 문장, 실제 라벨, 예측 라벨 출력
            f.write(f"<br>문장 {prediction['global_idx']}: {prediction['sentence']}<br>")
            f.write(f"실제 라벨: {label_meaning[prediction['true_label']]}<br>")
            f.write(f"예측 라벨: {label_meaning[prediction['predicted_label']]}<br>")
            f.write("확률:<br>")

            # 확률 출력
            for l, p in zip(label_list, prediction['probabilities']):
                f.write(f"  {l+1}({label_meaning[l]}): {p * 100:.2f}%<br>")

            # SHAP 분석 결과 출력 (문장 기반)
            f.write("<br>SHAP 분석 결과:<br>")
            text = shap.plots.text(shap_values[i], display=False)
            f.write(text)
            f.write("<br>")

        with open("起来보어 검증데이터에서의 이상치 분석.html", "r", encoding='UTF-8') as f:
            html_content = f.read()

            shap_label_meaning = {
    "LABEL_0": "Directional",
    "LABEL_1": "Resultative",
    "LABEL_2": "Completive",
    "LABEL_3": "Inchoative",
    "LABEL_4": "Discourse"
}

            # 라벨을 대체
            for key, value in shap_label_meaning.items():
                html_content = html_content.replace(key, value)

            # 변경된 내용을 다시 HTML 파일에 저장
        with open("起来보어 검증데이터에서의 이상치 분석.html", "w", encoding='UTF-8') as f:
           f.write(html_content)



In [27]:
analyze_outlier_probabilities(validation_outlier_indices,
                              validation_dataloader,
                              validation_sentences,
                              shap_values)

## 테스트 데이터셋에 대한 시각화

In [28]:
test = 'qilai_test_data.txt'

## 테스트 세트 문장, 라벨, 길이
test_sentences,test_labels,test_len=load_data(test)
test_labels_to_int=[int(label)-1 for label in test_labels]
test_sentences_mask_replaced=preprocess(test_sentences,'起来')

print(f"테스트 세트 문장 개수: {test_len}")

# === 테스트 세트에 대한 토큰화 작업 ===
test_sentences_tokenized = [tokenizer.tokenize(sen) for sen in test_sentences_mask_replaced]
test_sentences_token_ids = pad_sequences(
    [tokenizer.convert_tokens_to_ids(token_sen) for token_sen in test_sentences_tokenized],
    dtype='long', truncating='post', padding='post'
)

print(f"테스트 데이터셋 크기: {len(test_sentences_token_ids)}")

# === 테스트 데이터에 대한 어텐션 마스크 생성 ===
test_attention_masks = create_attention_masks(test_sentences_token_ids)

print(f"테스트 데이터 어텐션 마스크 크기: {len(test_attention_masks)}")

# 테스트 세트
test_datas = torch.tensor(test_sentences_token_ids)  # 테스트 문장 데이터
test_labels = torch.tensor(test_labels_to_int)       # 테스트 라벨 데이터
test_masks = torch.tensor(test_attention_masks)      # 테스트 어텐션 마스크 데이터

print(f"테스트 데이터 텐서 크기: {test_datas.size()}")

# === 테스트 데이터 ===
test_data = TensorDataset(test_datas, test_masks, test_labels)
test_sampler = SequentialSampler(test_data)  # 순차 샘플링
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)

print(f"테스트 DataLoader 크기: {len(test_dataloader)}")

테스트 세트 문장 개수: 50
테스트 데이터셋 크기: 50
테스트 데이터 어텐션 마스크 크기: 50
테스트 데이터 텐서 크기: torch.Size([50, 35])
테스트 DataLoader 크기: 5


In [29]:
# 테스트 데이터에서 임베딩 추출
model.eval()  # 모델을 평가 모드로 전환

test_embeddings = []
test_labels_list = []
test_sentences_list = []

for batch in test_dataloader:
    batch_token_ids, batch_mask, batch_labels = [t.to(device) for t in batch]

    with torch.no_grad():
        outputs = model(batch_token_ids, attention_mask=batch_mask, output_hidden_states=True)

    hidden_states = outputs.hidden_states

    if hidden_states is not None:
        # [CLS] 토큰의 임베딩 추출
        cls_embeddings = hidden_states[-1][:, 0, :].cpu().numpy()
        test_embeddings.extend(cls_embeddings)
        test_labels_list.extend(batch_labels.cpu().numpy())
        test_sentences_list.extend([f"Test Sentence {i}" for i in range(len(batch_labels))])

# NumPy 배열로 변환
test_embeddings = np.array(test_embeddings)
test_labels_list = np.array(test_labels_list)


In [30]:
# 테스트 데이터에 대해 t-SNE 적용
tsne = TSNE(n_components=2, random_state=42)
tsne_test_embeddings = tsne.fit_transform(test_embeddings)


In [31]:
len(tsne_test_embeddings),len(test_labels_list)

(50, 50)

In [64]:
test_sentences_mask_replaced

['随后,她把女儿抱起来,放到自己的腿上。',
 '这是一个大石块自然叠起来的洞子，一人多高，越往里越窄越低。',
 '他的话还没说完，看热闹的人都哗的一声大笑起来。',
 '小小笑起来，满口的小牙可白可整齐啦。',
 '说完,站起来,作出宽怀大量的样子,一瘸一瘸走了。',
 '听完，他马上和老婆一样高兴起来。',
 '想到这个碴，小林兴奋起来，立即骑上车去找修车老头。',
 '很大方地点着，与小林一人一支，抽了起来。',
 '女儿一高兴，全家情绪又都好起来。',
 '这时小林头脑清醒过来，不再管梦，赶忙爬起来去排队买豆腐。',
 '掌板的同伴将他的板子举了起来。',
 '他扔掉烟屁股，站起来，从同伴手中拿过一杆唢呐。',
 '人们又紧张起来，纷纷说：要打鼓了，要打鼓了。',
 '角度又选得那样好，太阳从他们的后背升起来。',
 '女人的心不由得开始砰砰跳起来。',
 '男人梦一样的声音，又悄悄在她耳边响起来了。',
 '上大学那时候，你对我说的话加起来也不会超过十句。',
 '他从沙发上站起来，走到窗前，入神地眺望着波光闪烁的哈得逊河。',
 '谁也不去注意那条临时挂起来的大红布标语。',
 '就在这时，车厢乱了起来。',
 '因此聊起来，常常是主角，说得大家个个儿腮胀。',
 '我把蛇挂起来，将皮剥下，不洗，放在案板上。',
 '把葱末、姜末和蒜末投进去，叫声：“吃起来！”',
 '不似刚才紧张，话也多起来了。',
 '燕窝这种东西，是海鸟叼来小鱼小虾，用口水粘起来的。',
 '大家看出是谁赢了，都高兴松动起来，盯着王一生看。',
 '那两个人赶紧站起来，连说可以。',
 '本想到人群里说说，但又止住了，随人们传吧，我开始高兴起来。',
 '忽然人群乱起来，纷纷闪开。',
 '我笑起来，想：不做俗人，哪儿会知道这般乐趣？',
 '亮道看起来光光明明平平坦坦，却拥挤不堪。',
 '女人重新痛哭起来。',
 '饿你三天，你就知道强盗的饭吃起来也香甜。',
 '我不要听，我要走，你放我走！”女人又哭泣起来。',
 '女人捂着脸呜呜哭泣起来，心里恨恨地嚷，“杀了他，杀了他！”',
 '但这紧要关口我却大哭大叫起来，这哭叫便改变了我的命运。',
 '现在回想起来，他们俩口对我十分疼爱，把我当亲儿子待，好东西先尽着我吃。',
 '那时嫌小，放进圈子养起来了，一年工夫就长成了

In [66]:
# Plotly 시각화를 위한 Figure 객체 생성
fig = go.Figure()

# 전체 테스트 데이터에 대해 문장 번호 생성
sentence_indices = np.arange(1, len(test_labels_list) + 1)

# 각 라벨에 대해 t-SNE 임베딩 데이터를 기반으로 산점도를 생성합니다.
for label in label_meaning.keys():
    # 현재 라벨에 해당하는 데이터 마스크 생성
    mask = test_labels_list == label

    # 각 라벨에 대해 산점도(Scatter) 생성
    scatter = go.Scatter(
        x=tsne_test_embeddings[mask, 0],  # 첫 번째 축의 임베딩 값
        y=tsne_test_embeddings[mask, 1],  # 두 번째 축의 임베딩 값
        mode='markers',  # 마커 모드로 설정

        marker=dict(
            size=5,  # 마커 크기
            color=label_colors[label],  # 각 라벨에 해당하는 색상 지정
            showscale=False  # 범례 제거
        ),
        # 전체 데이터의 인덱스에서 각 라벨의 필터링된 마스크에 해당하는 인덱스만 선택하여 문장 번호를 표시
        text=[f"{test_sentences_mask_replaced[idx-1]}" for idx in sentence_indices[mask]],
        hoverinfo='text',  # 툴팁에 표시될 정보의 유형 설정
        name=f"{label_meaning.get(label)}",  # 범례에 표시할 라벨 이름 설정
        visible=True  # 보이도록 설정
    )
    fig.add_trace(scatter)  # 산점도를 그래프에 추가

# 레이아웃 설정
fig.update_layout(
    title='t-SNE Sentence Embeddings of Test Data by Label',  # 그래프 제목 설정
    legend_title="Labels",  # 범례 제목 설정
    showlegend=True  # 범례를 보이도록 설정
)

# 그래프를 출력
fig.show()


간단한 실습